**Algorithm 4.3** Putting it all together

In [3]:
"""Paper referenced: https://services.math.duke.edu/~dasgupta/papers/comp.pdf"""
import sympy,math, gmpy2

# p = 3
# N = 4
# M = 10

%run Algo_4_2a.ipynb
%run Algo_4_2b.ipynb
%run Basis.ipynb


# f = fEqnlogApprox35(p,M).polynomial()
# integral_ii = integral_ii_dict(p,N,M)
# integral_iv = integral_iv_dict(p,N,M)
# integral_iii = integral_iii_dict(f,p,N,M)
# integral_i = integral_i_dict(f,p,N,M)

S = Qp(p, prec = M+1, type = 'capped-rel', print_mode = 'series')	#Defines the padic ring of rationals, with specified precision.
R.<y> = PowerSeriesRing(S)	#Allows us to write polynomials in y with coefficients in S.

**Step 1** Calculate ord_p u(alpha,tau)

In [6]:
def order_p(p,N,a,c):
    """D is just D_{1,1}. alpha denotes the equation N[1]-[N]"""
    order = -N*genDedekind(1,1,a,N*c)+genDedekind(1,1,a,c)
    if order == round(order):
        return int(order)
    return 12*order

# for a in range(11):
#     for c in range(4):
#         print(order_p(p,N,a,c))
# print(order_p(3,4,1567,928*2))

63


**Step 2** Calculate log_beta 

In [ ]:
def rhsOfEqn44(a,c,u,v,p,N):
    """Prop 3.2 but s=1
    Checked: It always returns an integer.
    Returns u([infty] - [a/Nc])"""
    total = 0 
    for l in range(N*c):
        total -= BernPolyTilde(1, a*(l+v/p)/(N*c)-u/p)*(-BernPolyTilde(1,1/c*(l+v/p))+N*BernPolyTilde(1,(l+v/p)/(N*c)))
    return total*12

def log_betaHelper(table,x):
    """Finds discrete logarithm of x with base beta """
    for index in range(len(table)):
        if (table[index]-x).valuation()>0:
            return index
    raise "error"

def log_beta(a,c,p,beta,tau,N):
    """Returns log_beta"""
    #Construct discrete logarithm table of beta
    table = [1]
    for i in range(1,p^2-1): 
        table.append(table[i-1]*beta)
    #Calculate final value
    total = 0
    for u in range(p):
        for v in range(p):
            if u == 0 and v == 0:
                pass
            else:
                total += log_betaHelper(table,T(u-v*tau))*rhsOfEqn44(a,c,u,v,p,N)
    #Calculate remainder (mod p^2-1)
    Zp2minus1 = Integers(p^2-1)
    return int(Zp2minus1(total))

# log_beta(32,2,p,beta,tau,N)

**Step 3** Calculate log_p

In [ ]:
def sumOfMeasures(i,p,a,c,N):
    #finds the sum of measures u_bar_m(i+pZp)
    total = 0
    Modp = IntegerModRing(p)
    for y in range(1,p):
        for x in range(p):
            if (x*(Modp(y)^-1)-i)%p == 0:
                total += rhsOfEqn44(a,c,x,y,p,N)
    return total

def log_pHelper(tau,i,integral_i,integral_ii,integral_iii,integral_iv):
    """i is the basis m_i, which is [inf] - [i/N] in our case. 
    Returns integral in RHS of 45. Here x-y*tau replaces x-y*gamma^-1tau?"""
    first = T(integral_i[i])
    third = T(integral_iii[i])
    second = 0 
    for j in range(p):
        #first part of eqn 36
        second += log(tau-j)*sumOfMeasures(j,p,i,1,N)  
        #second part of eqn 36
        for n in range(1,M):
            second -= T(integral_ii[(i,j,n)]/n)/(tau-j)^n
    forth = 0 
    for n in range(1,M):
        #Should there be a /n here? 
        forth -= T(tau^n)*T(integral_iv[(i,n)]/n)
    return first + second + third + forth

# log_pHelper(tau_p,2,integral_i,integral_ii,integral_iii,integral_iv)

def log_p(a,c,p,N,tau,integral_i,integral_ii,integral_iii,integral_iv):
    gammas = getGammas(a,c*N)
    #a_gamma,i can be assume to be 1 throughout, since we will iterate over all gammas
    total = 0
    for gamma in gammas[:-1]:
        ga, gb, gc, gd = gamma.a, gamma.b, gamma.c, gamma.d 
        base = gamma.x
        gammaInv_tau = (gd*tau-gb)/(-gc*tau+ga)
        total += gamma.coeff*log_pHelper(gammaInv_tau,base,integral_i,integral_ii,integral_iii,integral_iv)
    return total 

# log_p(48599,2820/5,p,N,tau,integral_i,integral_ii,integral_iii,integral_iv)

**Step 4** Define u(alpha,tau)

In [ ]:
def u_alpha_tau(a,c,p,N,tau,beta,integral_i,integral_ii,integral_iii,integral_iv): 
    exp = log_p(a,c,p,N,tau,integral_i,integral_ii,integral_iii,integral_iv).exp()
    ord_p = order_p(p,N,a,c)
    log_b = log_beta(a,c,p,beta,tau,N)
    return T(p^ord_p) * beta^log_b * exp